In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/divorce-prediction/divorce_data.csv', sep=';')

In [ ]:
X = data.drop('Divorce', axis=1)
y = data.loc[:, 'Divorce']

In [ ]:
data.info()

there no need to encode since there is no columns based on object type && also no need to filling nan values since all the columns non-null

In [ ]:
data.describe().T

there is no need to scale any of columns since they all between 0 and 4, and also has standard deviation smaller than 2

In [ ]:
hist = data.hist(figsize=(40, 30))

there no need to cut any value because none of them seem higher or lower than others

In [ ]:
data.corr()['Divorce'].drop('Divorce').sort_values(ascending=False)

there is no column that affects our target column too low, they all feed our target goodly (the lowest term is 0.42 which means exactly perfect)

---

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=.3, random_state=777)

let's create our train and see whether we can earn a good score without any kind of preprocess

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=2500)
model.fit(Xtrain, ytrain.values.flatten())
ypred = model.predict(Xvalid)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(yvalid, ypred), confusion_matrix(yvalid, ypred)

let's add some new features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 3))
columns = X.columns
X = pd.DataFrame(scaler.fit_transform(X), columns=columns)

In [ ]:
info = data.corr()['Divorce'].drop('Divorce').sort_values(ascending=False)
upper_columns = list(info[:5].index)

columns = list(X.columns)
from sklearn.preprocessing import MinMaxScaler
X = pd.DataFrame(MinMaxScaler(feature_range=(1, 3)).fit_transform(X), columns=columns)

new_arrays = np.zeros([X.shape[0], 1])
for i in upper_columns:
    new_arrays = np.concatenate([new_arrays, (X[i] ** 2).values.reshape(-1, 1)], axis=1)
for i in upper_columns:
    for j in upper_columns:
        if i != j:
            new_arrays = np.concatenate([new_arrays, (X[i] * X[j]).values.reshape(-1, 1)], axis=1)

new_arrays = new_arrays[:, 1:]
X = pd.concat([pd.DataFrame(X), pd.DataFrame(new_arrays)], axis=1)

and let's reduct the dimension to avoid overfit by using KernelPCA

In [ ]:
components = round(X.shape[1] / 4)
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(kernel='rbf', n_components=components)
kpca.fit(X)
X = kpca.transform(X)

In [ ]:
Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=.3, random_state=777)

model = XGBClassifier(n_estimators=2500)
model.fit(Xtrain, ytrain.values.flatten())
ypred = model.predict(Xvalid)

accuracy_score(yvalid, ypred), confusion_matrix(yvalid, ypred)

now we've earnt a good value of accuracy. now let's test it in kfolds

In [ ]:
from sklearn.model_selection import cross_val_score
kfold_results = cross_val_score(estimator=XGBClassifier(n_estimators=2500), X=X, y=y, scoring='accuracy', cv=5, n_jobs=-1)

In [ ]:
kfold_results.mean()

---